<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>

# Quickstart: Run a Full Simulation
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/00_Quickstart.ipynb)

**Goal:** run an end-to-end simulation and get a results CSV, audit trail, baseline comparison, and PDF report.

**You will learn:**
- Load a preset scenario
- Apply a demo-safe patient profile
- Run the full pipeline with `iints.run_full`
- Locate the output files

**Expected output:** a `results/quickstart` folder with CSV + audit + baseline + PDF.


In [1]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


## Step 1: Load a preset and algorithm


In [2]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


## Step 2: Apply a demo‑safe patient profile


In [3]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


## Step 3: Run full pipeline (audit + PDF + baseline)


In [4]:
outputs = iints.run_full(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=360,
    time_step=preset["time_step_minutes"],
    seed=42,
    output_dir="results/quickstart",
    enable_profiling=True,
)

outputs.keys()


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


dict_keys(['results', 'safety_report', 'baseline_comparison', 'results_csv', 'audit', 'baseline_files', 'report_pdf'])

## Step 4: Inspect outputs


In [5]:
outputs["results"].head()


,time_minutes,glucose_actual_mgdl,glucose_to_algo_mgdl,delivered_insulin_units,algo_recommended_insulin_units,sensor_status,pump_status,pump_reason,basal_insulin_units,bolus_insulin_units,...,safety_level,safety_actions,safety_reason,safety_triggered,supervisor_latency_ms,human_intervention,human_intervention_note,algorithm_why_log,algorithm_latency_ms,step_latency_ms
0,0,150.000000,150.000000,0.033333,0.033333,ok,ok,approved,0.033333,0.0,...,safe,,APPROVED,False,0.012414,False,,[],0.001463,0.054942
1,5,142.495556,142.495556,0.033333,0.033333,ok,ok,approved,0.033333,0.0,...,safe,,APPROVED,False,0.009358,False,,[],0.001362,0.051416
2,10,135.357444,135.357444,0.033333,0.033333,ok,ok,approved,0.033333,0.0,...,safe,,APPROVED,False,0.008215,False,,[],0.000961,0.050144
3,15,128.562906,128.562906,0.033333,0.033333,ok,ok,approved,0.033333,0.0,...,safe,,APPROVED,False,0.008446,False,,[],0.000892,0.030086
4,20,122.090316,122.090316,0.033333,0.033333,ok,ok,approved,0.033333,0.0,...,safe,,APPROVED,False,0.006803,False,,[],0.000901,0.025658


In [6]:
print("Report PDF:", outputs.get("report_pdf"))
print("Audit summary:", outputs.get("audit", {}).get("summary"))


Report PDF: results/quickstart/clinical_report.pdf
Audit summary: results/quickstart/audit/audit_summary.json


### Recap
You now have a results CSV, an audit trail, a baseline comparison, and a PDF report in `results/quickstart/`.
